## Universal Sentence Encoder (trainable)

[Исходный гайд](https://amitness.com/2020/02/tensorflow-hub-for-transfer-learning/)

In [ ]:
import pandas as pd
df = pd.read_csv('http://bit.ly/clickbait-data')
df.head()

,title,label
0,"15 Highly Important Questions About Adulthood,...",1
1,250 Nuns Just Cycled All The Way From Kathmand...,1
2,"Australian comedians ""could have been shot"" du...",0
3,Lycos launches screensaver to increase spammer...,0
4,Fußball-Bundesliga 2008–09: Goalkeeper Butt si...,0


In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(df['title'], 
                                                    df['label'], 
                                                    test_size=0.3, 
                                                    stratify=df['label'], 
                                                    random_state=42)

In [ ]:
import tensorflow_hub as hub

encoder = hub.load('https://tfhub.dev/google/universal-sentence-encoder/4')
print(type(encoder(['Hello World'])))

(1, 512)


In [ ]:
print(tf.cast(encoder(['Hello World']), dtype=tf.float64))

tf.Tensor(
[[-2.60742530e-02 -8.46000239e-02 -2.67866030e-02  5.67842238e-02
   6.19704649e-02  3.82260047e-02  2.01149415e-02  2.74087563e-02
   8.69832113e-02  3.07918098e-02  4.10411879e-02  2.55183522e-02
   9.65159677e-04  5.89279756e-02  4.00954075e-02  4.67089228e-02
  -3.31279486e-02  4.07041796e-02 -1.19929593e-02 -4.76170778e-02
  -8.29666853e-03  7.05467537e-02  4.84353816e-03  8.09824839e-02
  -5.70177324e-02 -6.43551424e-02  6.97147325e-02 -6.98838457e-02
   2.64606159e-02 -2.35220436e-02  5.52375205e-02  4.91364114e-02
   5.69897145e-02 -6.10719398e-02  3.60851213e-02  2.32446212e-02
   5.60175218e-02 -1.09551847e-02  4.21308428e-02 -1.76954269e-02
   7.12432154e-03  2.04703286e-02 -7.10126385e-02 -8.45577493e-02
   4.36058305e-02  4.79397625e-02 -9.01802182e-02 -6.87343925e-02
   1.27959941e-02  1.62498821e-02 -4.83131073e-02  3.08832061e-02
   7.43833045e-03  3.39155607e-02  3.32343951e-02  4.33182493e-02
   6.36333823e-02  4.92026955e-02 -2.83108559e-02  8.13513435e-03

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

In [ ]:
model = tf.keras.models.Sequential()

In [ ]:
model.add(hub.KerasLayer('https://tfhub.dev/google/universal-sentence-encoder/4', 
                        input_shape=[], 
                        dtype=tf.string, 
                        trainable=True))

In [ ]:
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

In [ ]:
x = tf.keras.layers.Input(shape=[], dtype=tf.string)
y = hub.KerasLayer('https://tfhub.dev/google/universal-sentence-encoder/4', 
                    trainable=True)(x)
z = tf.keras.layers.Dense(1, activation='sigmoid')(y)
model = tf.keras.models.Model(x, z)

In [ ]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None,)]                 0         
_________________________________________________________________
keras_layer_1 (KerasLayer)   (None, 512)               256797824 
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 513       
Total params: 256,798,337
Trainable params: 256,798,337
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

In [ ]:
print(type(x_train))
x_train.head

<class 'pandas.core.series.Series'>


<bound method NDFrame.head of 30111    22 Things You Totally Forgot You Used To Do On...
19216       Sales Improve, but Profit Falls 9% at Walgreen
21857            19 Ways To Make A Next-Level Reading Nook
29826    Monday's no-confidence vote may signal end of ...
1631              Do You Know The Disney Movie By The Food
                               ...                        
14525                 Pigskin A Blanket: NFL Week 15 Picks
11252    17 Times The Internet Perfectly Summed Up Bein...
9698     Swedish wrestler throws away Olympic bronze me...
23604    Greenpeace activists clash with Japanese whali...
31075    27 Tweets About Being Latino That Are Way Too ...
Name: title, Length: 22390, dtype: object>

In [ ]:
model.fit(x_train, 
          y_train, 
          epochs=2, 
          validation_data=(x_test, y_test))

Epoch 1/2
700/700 [==============================] - 108s 155ms/step - loss: 0.0973 - accuracy: 0.9723 - val_loss: 0.0569 - val_accuracy: 0.9807
Epoch 2/2
700/700 [==============================] - 108s 154ms/step - loss: 0.0122 - accuracy: 0.9970 - val_loss: 0.0525 - val_accuracy: 0.9840


In [ ]:
# Clickbait
print(model.predict(["21 Pictures That Will Make You Feel Like You're 99 Years Old"]))

# Not Clickbait
print(model.predict(['Google announces TensorFlow 2.0']))

[[0.9984434]]
[[0.00692841]]
